In [1]:
from BMI_HBVmountain_Python import *
from Calibration_BRC import *
import os
import netCDF4 as nc

In [10]:
import cma
import hashlib
import time
import warnings
from pathlib import Path
# from cartopy.io import shapereader
from cftime import date2num
from matplotlib.ticker import MaxNLocator
import logging
logging.basicConfig(level=logging.WARNING)
warnings.filterwarnings("ignore", category=UserWarning)
from cma.fitness_transformations import EvalParallel2


# from cartopy.io import shapereader
# from ruamel.yaml import YAML
# import ewatercycle.models
# from ewatercycle.config import CFG
# from ewatercycle.forcing import load_foreign
# from ewatercycle.observation.grdc import get_grdc_data
# from ewatercycle.util import get_time

In [11]:
POPSIZE = 9  # it can be equal to number of available cores * 0.75
MAXITER = 10  # with this, the notebook takes ~4 hours (maximum)

In [ ]:
def run_model(parameters):
    """Setup and run model."""
    model = BMI_HBVmountain()
    
    # Generate a random tag for work_dir name
    # Each work_dir is used by one worker in multiprocessing
#    tag = hashlib.md5(str(parameters).encode('utf-8')).hexdigest()
#    work_dir = CFG['output_dir'] / f"marrmot-m14_{tag}"
        
    cfg_file, cfg_dir = model.setup(bare_parameters=  Parameters(parameters[0][0], parameters[0][1], parameters[0][2], parameters[0][3], parameters[0][4],
                                                                 parameters[0][5], parameters[0][6], parameters[0][7], parameters[0][8], parameters[0][9]),
                                    forest_parameters=Parameters(parameters[1][0], parameters[1][1], parameters[1][2], parameters[1][3], parameters[1][4],
                                                                 parameters[1][5], parameters[1][6], parameters[1][7], parameters[1][8], parameters[1][9]), 
                                    grass_parameters= Parameters(parameters[2][0], parameters[2][1], parameters[2][2], parameters[2][3], parameters[2][4],
                                                                 parameters[2][5], parameters[2][6], parameters[2][7], parameters[2][8], parameters[2][9]),
                                    rip_parameters=   Parameters(parameters[3][0], parameters[3][1], parameters[3][2], parameters[3][3], parameters[3][4],
                                                                 parameters[3][5], parameters[3][6], parameters[3][7], parameters[3][8], parameters[3][9]), 
                                    slow_parameters=  Slow_Paramters(parameters[4][0], parameters[4][1])
    )
    
    # Initialize the model inside the docker container through GRPC4bmi
    model.initialize(cfg_file)
    
    # Convert start to date number to be used in if statement
    start_time = date2num(
        get_time(CALIBRATION['start']),
        model.time_units
    )
    simulated_discharge = []
    end_time = model.end_time
    # Perform all timesteps of the model, update output fields
    while (model.time < end_time):
        model.update()
        # Store model time and variable output after the spin up period
        if model.time >= start_time:
            simulated_discharge.append(model.get_value('flux_out_Q')[0])
    model.finalize()
    return np.array(simulated_discharge)


def get_catchment_area(catchment):
    """Get the catchment area in m2."""
    shapefile = FORCING.shape
    shape = shapereader.Reader(shapefile)
    attr = next(shape.records())
    return attr.attributes['SUB_AREA'] * 1e6


def get_observations(station_id):
    """Get GRDC observation data."""
    observations, metadata = get_grdc_data(station_id, CALIBRATION["start"], CALIBRATION["end"])
    return observations['streamflow']


def objective_function(parameters, observation, area):
    """Calculate objective function.

    Runs the model, converts the output to GRDC streamflow units
    and calculates NSE from simulation data with observation data.
    This is the function that is going to be optimized by scipy.brute.
    """
    simulation = run_model(parameters)
    # flux_out_Q unit conversion factor from mm/day to m3/s
    conversion = 1 / (1000 * 86400)
    # convert simulated streamflow to same units as observation
    simulation = simulation * area * conversion
    nse_value = hm.nse(simulation, observation)
    return 1 - nse_value


def transform(scaled_parameters):
    """Transforms the scaled_parameter to parameter.

    if x = scaled_parameter and y = parameter,
    then x is in the range [0, 10] and y is in the range [a, b].
    To map the values [0, 10] into [a, b],
    we use the transformations a + (b-a) * x/10.
    For more information on this transformation,
    please see
    http://cma.gforge.inria.fr/cmaes_sourcecode_page.html#practical
    """
    parameters = []
    for scaled_parameter, bound in zip(scaled_parameters, PARAMETERS_BOUNDS):
        scale = (bound[1] - bound[0]) / 10.0
        parameter = bound[0] + (scale * scaled_parameter)
        parameters.append(parameter)
    return parameters


def wrapped_objective_function(scaled_parameters, *args):
    """A wrapper around the objective function.

    The wrapper transforms the scaled_parameters before
    the actual function call.
    """
    parameters = transform(scaled_parameters)
    return objective_function(parameters, *args)


def run_calibration(parameter_bounds, observation, area):
    """Return the optimum parameters found by CMA-ES method."""
    # Set some options for optimization needed when multiprocessing
    options = cma.CMAOptions()
    options.set({
        'bounds': [0, 10],  # for scaled parameters
        'seed': 1234,  # set a seed to reproduce results
        'verbose': -9,  # verbosity of initial/final message: maximally quiet
        'popsize': POPSIZE,
        'maxiter': MAXITER,
        'tolfun': 1e-17,  # very small value due to model creeping behavior
        'tolx': 1e-5,
        'verb_filenameprefix': str(CFG['output_dir']),  # cma-es output path
    })

    no_of_variables = len(parameter_bounds)
    # initial mean value and standard deviation
    x0 = 5.0
    sigma0 = 2.0

    # Initialize the CMA-ES
    cma_es = cma.CMAEvolutionStrategy(no_of_variables * [x0], sigma0, options)

    # Store the results of each iteration
    all_scores = []
    all_parameters = []
    
    # Use parallel processing
    with EvalParallel2(number_of_processes=options['popsize']) as evaluations:
        # Use ask-and-tell interface
        while not cma_es.stop():
            solutions = cma_es.ask()
            cma_es.tell(
                solutions,
                evaluations(
                    solutions,
                    fitness_function=wrapped_objective_function,
                    args=(observation, area)
                ),
            )
            # Use transform to return parameters and not scaled ones
            all_parameters.append(transform(cma_es.best.last.x))
            all_scores.append(cma_es.best.last.f)
    print(f"---- CMA-ES stopped due to: {cma_es.stop()} ----")

    # Make full output
    full_output = (transform(cma_es.result.xbest),
                   cma_es.result.fbest,
                   all_parameters,
                   all_scores)
    return full_output